In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [69]:
numEpocas = 1000     # Número de épocas.

tamanho = pd.read_csv('winequality-red.csv')
q = len(tamanho)  # Número de amostras

eta = 0.25            # Taxa de aprendizado

m = 11                # Número de neurônios na camada de entrada 
N = [15,10,5]         # Número de neurônios em cada camada escondida. A professora pediu minimo 3
L = 1                 # Número de neurônios na camada de saída. (-1 = Vinho ruim E 1 = Vinho bom)

In [70]:
# Dados

FixedAcidity = pd.read_csv('winequality-red.csv', usecols=[0])
FixedAcidity = np.array(FixedAcidity)
FixedAcidity = np.transpose(FixedAcidity)

VolatileAcidity = pd.read_csv('winequality-red.csv', usecols=[1])
VolatileAcidity = np.array(VolatileAcidity)
VolatileAcidity = np.transpose(VolatileAcidity)

CitricAcid = pd.read_csv('winequality-red.csv', usecols=[2])
CitricAcid = np.array(CitricAcid)
CitricAcid = np.transpose(CitricAcid)

ResidualSugar = pd.read_csv('winequality-red.csv', usecols=[3])
ResidualSugar = np.array(ResidualSugar)
ResidualSugar = np.transpose(ResidualSugar)

Chlorides = pd.read_csv('winequality-red.csv', usecols=[4])
Chlorides = np.array(Chlorides)
Chlorides = np.transpose(Chlorides)

FreeSulfurDioxide = pd.read_csv('winequality-red.csv', usecols=[5])
FreeSulfurDioxide = np.array(FreeSulfurDioxide)
FreeSulfurDioxide = np.transpose(FreeSulfurDioxide)

TotalSulfurDioxide = pd.read_csv('winequality-red.csv', usecols=[6])
TotalSulfurDioxide = np.array(TotalSulfurDioxide)
TotalSulfurDioxide = np.transpose(TotalSulfurDioxide)

Density = pd.read_csv('winequality-red.csv', usecols=[7])
Density = np.array(Density)
Density = np.transpose(Density)

pH = pd.read_csv('winequality-red.csv', usecols=[8])
pH = np.array(pH)
pH = np.transpose(pH)

Sulphates = pd.read_csv('winequality-red.csv', usecols=[9])
Sulphates = np.array(Sulphates)
Sulphates = np.transpose(Sulphates)

Alcohol = pd.read_csv('winequality-red.csv', usecols=[10])
Alcohol = np.array(Alcohol)
Alcohol = np.transpose(Alcohol)

Quality = pd.read_csv('winequality-red.csv', usecols=[11])
Quality = np.array(Quality)
Quality = np.transpose(Quality)


In [71]:
# Entrada do perceptron
X = np.vstack((FixedAcidity, VolatileAcidity, CitricAcid, ResidualSugar, 
               Chlorides, FreeSulfurDioxide, TotalSulfurDioxide, Density, 
               pH, Sulphates, Alcohol))

# Resultado para saida
d = np.array([1 if q >= 6 else -1 for q in Quality.flatten()])

# Bias
bias = 1

W = np.zeros([1, 12]) # vetor de pesos para 11 entradas e bias   # N ENTENDI A DIFERENÇA DESSE PARA O 'e', Q ESTA LOGO ABAIXO

E = np.zeros(q) # array de erros
Etm = np.zeros(numEpocas)

# Matrizes de pesos
W1 = np.random.random((N[0], m + 1))       #1 camada
W2 = np.random.random((N[1], N[0] + 1))    #2 camada
W3 = np.random.random((N[2], N[1] + 1))    #3 camada
W4 = np.random.random((L, N[2] + 1))       #saida


# Funcoes
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

def relu(x):
    return np.maximum(0, x)

def deriv_relu(x):
    return np.where(x > 0, 1, 0)

# caso queira testar com outra função só trocar aqui embaixo
funcao_ativacao = sigmoid  
derivada_funcao = deriv_sigmoid  


In [ ]:
# ===============================================================
# TREINAMENTO.
# ===============================================================

for i in range(numEpocas): # repete conforme num de epococas
    for j in range(q):
        
        Xb = np.hstack((bias, X[:, j])) # empilhando junto com o bias

        # camada oculta 1
        o1 = funcao_ativacao(np.dot(W1, Xb))  
        o1b = np.insert(o1, 0, bias)

        # camada oculta 2
        o2 = funcao_ativacao(np.dot(W2, o1b))
        o2b = np.insert(o2, 0, bias)

        # camada oculta 3
        o3 = funcao_ativacao(np.dot(W3, o2b))  
        o3b = np.insert(o3, 0, bias)
        
        # Saida
        Y = funcao_ativacao(np.dot(W4, o3b))

        # Erro 
        e = d[j] - Y  

        # Erro total
        E[j] = (e.transpose().dot(e))/2  

        delta4 = e * derivada_funcao(Y)
        vdelta3 = W4.T.dot(delta4)
        delta3 = vdelta3[1:] * derivada_funcao(o3)

        vdelta2 = W3.T.dot(delta3)
        delta2 = vdelta2[1:] * derivada_funcao(o2)

        vdelta1 = W2.T.dot(delta2)
        delta1 = vdelta1[1:] * derivada_funcao(o1)

        # Atualização dos pesos.
        W4 += eta * np.outer(delta4, o3b)
        W3 += eta * np.outer(delta3, o2b)
        W2 += eta * np.outer(delta2, o1b)
        W1 += eta * np.outer(delta1, Xb)

    # Imprime o número da época e o Erro Total.
    print('i = ' + str(i) + '   E = ' + str(E))
    Etm[i] = E.mean()

plt.xlabel("Épocas")
plt.ylabel("Erro Médio")
plt.plot(Etm, color='c')
plt.plot(Etm)
plt.show()    
  

In [ ]:
#################
# TESTE DA REDE #
#################

Error_Test = np.zeros(q)

for i in range(q):
    # Insere o bias no vetor de entrada.
    Xb = np.hstack((bias, X[:,i]))

    # camada oculta 1
    o1 = funcao_ativacao(np.dot(W1, Xb))  
    o1b = np.insert(o1, 0, bias)

    # camada oculta 2
    o2 = funcao_ativacao(np.dot(W2, o1b))
    o2b = np.insert(o2, 0, bias)

    # camada oculta 3
    o3 = funcao_ativacao(np.dot(W3, o2b))  
    o3b = np.insert(o3, 0, bias)
    
    # Saida
    Y = funcao_ativacao(np.dot(W4, o3b))
    
    Error_Test[i] = d[i] - Y.item()
    
print(Error_Test)
# Error_Test_aux = np.round(Error_Test) - d
# print(Error_Test_aux[0:6]) # criei esses para printar o array completo em duas partes,
# print(Error_Test_aux[6:12]) # pois não aparecia inteiro de uma vez só
print(np.round(Error_Test) - d) #aqui se ela acertou todas o vetor tem que estar zerado

